# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import desc, sum, udf, col, mean
from pyspark.sql.types import IntegerType

spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [2]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [3]:
user_log.filter(user_log['userId'] == '').select(['page']).dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [4]:
user_log.filter(user_log['userId'] == '').show(n=5)

+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|artist|      auth|firstName|gender|itemInSession|lastName|length|level|location|method| page|registration|sessionId|song|status|           ts|userAgent|userId|
+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|  null|Logged Out|     null|  null|            0|    null|  null| free|    null|   PUT|Login|        null|     5598|null|   307|1513721196284|     null|      |
|  null|Logged Out|     null|  null|           26|    null|  null| paid|    null|   GET| Home|        null|      428|null|   200|1513721274284|     null|      |
|  null|Logged Out|     null|  null|            5|    null|  null| free|    null|   GET| Home|        null|     2941|null|   200|1513722009284|     null|      |
|  null|Logged Out|     null|  nul

# Question 3

How many female users do we have in the data set?

In [5]:
user_log.filter(user_log['gender'] == 'F').select(['userId']).dropDuplicates().count()

462

# Question 4

How many songs were played from the most played artist?

In [6]:
user_log.groupby(user_log['artist']).count().sort(desc('count')).show(n=5)

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|                null| 1653|
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
+--------------------+-----+
only showing top 5 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [7]:
is_home = udf(lambda x: int(x == 'Home'), IntegerType())
user_log_home = user_log.withColumn('home', is_home(col('page')))

window = Window.partitionBy(['userId']).orderBy(desc('ts')).rangeBetween(Window.unboundedPreceding, 0)

user_log_rank = user_log_home.withColumn('rank', sum('home').over(window))
user_log_rank.select(['userId', 'page', 'rank']).show()

user_log_count = user_log_rank.where(user_log_rank.page == 'NextSong').groupby('userId', 'rank').count()
user_log_count.show()

user_log_count.agg({'count': 'avg'}).show()

+------+---------+----+
|userId|     page|rank|
+------+---------+----+
|  1436| NextSong|   0|
|  1436| NextSong|   0|
|  2088|     Home|   1|
|  2088| NextSong|   1|
|  2088| NextSong|   1|
|  2088| NextSong|   1|
|  2088| NextSong|   1|
|  2088| NextSong|   1|
|  2088| NextSong|   1|
|  2088| NextSong|   1|
|  2088| NextSong|   1|
|  2088| NextSong|   1|
|  2088| NextSong|   1|
|  2088| NextSong|   1|
|  2088| NextSong|   1|
|  2088| NextSong|   1|
|  2162|Downgrade|   0|
|  2162| NextSong|   0|
|  2162| NextSong|   0|
|  2162| NextSong|   0|
+------+---------+----+
only showing top 20 rows

+------+----+-----+
|userId|rank|count|
+------+----+-----+
|  1436|   0|    2|
|  2088|   1|   13|
|  2162|   0|   19|
|  2162|   2|   15|
|  2294|   0|    4|
|  2294|   1|   17|
|  2294|   2|    3|
|  2294|   3|   16|
|  2294|   4|    4|
|  2294|   5|   11|
|  2904|   0|    1|
|   691|   1|    3|
|  1394|   0|    9|
|  1394|   1|   17|
|  2275|   0|    3|
|  2756|   0|    4|
|  2756|   2|    1